In [30]:
import requests
import hashlib
import time
import pandas as pd
from pandas import json_normalize

secret_token = '696a6f47212d93f88cef71238cfb3d1cd357905b3e34eabd2bbc49124553c129'

# Generate the Unix timestamp
timestamp = int(time.time())

# Construct the query string
query_string = f'time={timestamp}'

# Generate the hash
hash_string = secret_token + str(timestamp)
hash_value = hashlib.sha512(hash_string.encode()).hexdigest()

# Construct the URL with the query string and hash
url = f'https://api.fabdb.net/cards?{query_string}&hash={hash_value}'

parameters = {
    "page": 1,
    "per_page": 100
}

total_pages = 19

# Set the request headers
headers = {
    'Authorization': f'Bearer {secret_token}',
    'Accept': 'application/json'
}


In [31]:
data_records = []

for page in range(1, total_pages + 1):
    parameters['page'] = page
    
    response = requests.get(url, params=parameters)

    # process response
    data = response.json()

    # Extract the relevant data records from the response
    data_records.extend(data['data'])
pd.set_option('display.max_columns', None)
# Normalize the data before storing it in the dataframe
df2 = pd.json_normalize(data_records)
df2

,identifier,name,legality,keywords,text,rarity,image,printings,stats.resource,stats.life,stats.intellect,stats.attack,stats.defense,stats.cost,stats
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 863, 'language': 'en', 'name': 'Eye of...",3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,dash-inventor-extraordinaire,"Dash, Inventor Extraordinaire",[],"[mechanologist, hero]",You may start the game with a Mechanologist it...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 864, 'language': 'en', 'name': 'Dash, ...",NaN,40.0,4.0,NaN,NaN,NaN,NaN
2,dash,Dash,[],"[mechanologist, hero, young]",You may start the game with a Mechanologist it...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 865, 'language': 'en', 'name': 'Dash',...",NaN,20.0,4.0,NaN,NaN,NaN,NaN
3,teklo-plasma-pistol,Teklo Plasma Pistol,[],"[mechanologist, weapon, pistol, 2h]",**Action** - Remove a steam counter from Teklo...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 866, 'language': 'en', 'name': 'Teklo ...",NaN,NaN,NaN,2.0,NaN,NaN,NaN
4,teklo-foundry-heart,Teklo Foundry Heart,[],"[mechanologist, equipment, chest]",**Once per Turn Action** - [Resource]: Banish ...,L,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 867, 'language': 'en', 'name': 'Teklo ...",NaN,NaN,NaN,NaN,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,searing-emberblade,Searing Emberblade,[],"[draconic, ninja, weapon, sword, 2h]",**Once per Turn Action** - [2 resource]: **Att...,T,https://fabdb2.imgix.net/cards/printings/UPR04...,"[{'id': 12112, 'language': 'en', 'name': 'Sear...",NaN,NaN,NaN,3.0,NaN,NaN,NaN
1896,heat-wave,Heat Wave,[],"[draconic, ninja, equipment, arms]",**Instant** - Destroy Heat Wave: Phoenix Flame...,C,https://fabdb2.imgix.net/cards/printings/UPR04...,"[{'id': 12113, 'language': 'en', 'name': 'Heat...",NaN,NaN,NaN,NaN,0.0,NaN,NaN
1897,phoenix-form,Phoenix Form,[],"[draconic, ninja, action, attack]","If you control 1 or more Phoenix Flames, Phoen...",M,https://fabdb2.imgix.net/cards/printings/UPR04...,"[{'id': 12116, 'language': 'en', 'name': 'Phoe...",1.0,NaN,NaN,3.0,3.0,0.0,NaN
1898,spreading-flames,Spreading Flames,[],"[draconic, ninja, action, attack]",Draconic attacks you control have +1 [attack] ...,M,https://fabdb2.imgix.net/cards/printings/UPR04...,"[{'id': 12119, 'language': 'en', 'name': 'Spre...",1.0,NaN,NaN,3.0,2.0,1.0,NaN


In [32]:
# Explode the 'printings' column to expand the list
df_printings = df2.explode('printings')

# Expand the 'printings' column into separate columns
df_printings = pd.concat([df_printings['printings'].apply(pd.Series), df_printings], axis=1)

# Merge the expanded data with the original DataFrame
df_merged = pd.concat([df2.drop('printings', axis=1), df_printings], axis=1)

# Drop the original 'printings' column
df_merged.drop('printings', axis=1, inplace=True)
df_merged.head(2)

,identifier,name,legality,keywords,text,rarity,image,stats.resource,stats.life,stats.intellect,stats.attack,stats.defense,stats.cost,stats,id,language,name,text,flavour,sku,set,rarity,finish,edition,image,identifier,name,legality,keywords,text,rarity,image,stats.resource,stats.life,stats.intellect,stats.attack,stats.defense,stats.cost,stats
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,863,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...","{'sku': 'ARC000-CF', 'finish': 'cold', 'set': ...",Arcane Rising,F,Cold foil,{},https://fabdb2.imgix.net/cards/printings/ARC00...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1281,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...","{'sku': 'U-ARC000-RF', 'finish': 'rainbow', 's...",Arcane Rising Unlimited,F,Rainbow foil,{},https://fabdb2.imgix.net/cards/printings/U-ARC...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Use json_normalize to unnest the 'sku' column
df_sku = pd.json_normalize(df_merged['sku'])

# Reset the indexes of the DataFrames
df_merged.reset_index(drop=True, inplace=True)
df_sku.reset_index(drop=True, inplace=True)

# Combine the expanded 'sku' columns with the rest of the DataFrame
df_unnested = pd.concat([df_merged.drop('sku', axis=1), df_sku], axis=1)
df_unnested.head(2)

,identifier,name,legality,keywords,text,rarity,image,stats.resource,stats.life,stats.intellect,stats.attack,stats.defense,stats.cost,stats,id,language,name,text,flavour,set,rarity,finish,edition,image,identifier,name,legality,keywords,text,rarity,image,stats.resource,stats.life,stats.intellect,stats.attack,stats.defense,stats.cost,stats,sku,finish,number,set.id,set.name,set.released,set.browseable,set.draftable,set.0
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,863,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...",Arcane Rising,F,Cold foil,{},https://fabdb2.imgix.net/cards/printings/ARC00...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,ARC000-CF,cold,000,arc,Arcane Rising,2020-03-27,True,True,NaN
1,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1281,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...",Arcane Rising Unlimited,F,Rainbow foil,{},https://fabdb2.imgix.net/cards/printings/U-ARC...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,U-ARC000-RF,rainbow,000,arc,Arcane Rising,2020-03-27,True,True,NaN


In [34]:
mapping = {
    'stats.resource': 'pitch value',
    'stats.life': 'life',
    'stats.attack': 'attack',
    'stats.defense': 'defense',
    'stats.cost': 'cost',
    'stats.intellect': 'intellect'
}

# Apply the mapping function to rename the 'stats' columns
df_unnested.rename(columns=mapping, inplace=True)

# Display the DataFrame with updated column names
df_unnested.head(2)

,identifier,name,legality,keywords,text,rarity,image,pitch value,life,intellect,attack,defense,cost,stats,id,language,name,text,flavour,set,rarity,finish,edition,image,identifier,name,legality,keywords,text,rarity,image,pitch value,life,intellect,attack,defense,cost,stats,sku,finish,number,set.id,set.name,set.released,set.browseable,set.draftable,set.0
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,863,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...",Arcane Rising,F,Cold foil,{},https://fabdb2.imgix.net/cards/printings/ARC00...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,ARC000-CF,cold,000,arc,Arcane Rising,2020-03-27,True,True,NaN
1,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1281,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...",Arcane Rising Unlimited,F,Rainbow foil,{},https://fabdb2.imgix.net/cards/printings/U-ARC...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,U-ARC000-RF,rainbow,000,arc,Arcane Rising,2020-03-27,True,True,NaN


In [35]:
# Define the mapping dictionary for rarity
rarity_mapping = {
    'F': 'Fabled',
    'M': 'Majestic',
    'C': 'Common',
    'R': 'Rare',
    'L': 'Legendary'
}

# Use the replace method to update the 'rarity' column
df_unnested['rarity'] = df_unnested['rarity'].replace(rarity_mapping)

# Display the DataFrame with updated rarity values
df_unnested.head(2)

,identifier,name,legality,keywords,text,rarity,image,pitch value,life,intellect,attack,defense,cost,stats,id,language,name,text,flavour,set,rarity,finish,edition,image,identifier,name,legality,keywords,text,rarity,image,pitch value,life,intellect,attack,defense,cost,stats,sku,finish,number,set.id,set.name,set.released,set.browseable,set.draftable,set.0
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,Fabled,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,863,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...",Arcane Rising,Fabled,Cold foil,{},https://fabdb2.imgix.net/cards/printings/ARC00...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,Fabled,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,ARC000-CF,cold,000,arc,Arcane Rising,2020-03-27,True,True,NaN
1,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,Fabled,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1281,en,Eye of Ophidia,**Legendary** *(You may only have 1 Eye of Oph...,"Beyond the turbulent waters of Death’s Knell, ...",Arcane Rising Unlimited,Fabled,Rainbow foil,{},https://fabdb2.imgix.net/cards/printings/U-ARC...,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,Fabled,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,U-ARC000-RF,rainbow,000,arc,Arcane Rising,2020-03-27,True,True,NaN


In [36]:
df_unique = df_unnested.loc[:,~df_unnested.columns.duplicated()].copy()
df_unique.head(2)


,identifier,name,legality,keywords,text,rarity,image,pitch value,life,intellect,attack,defense,cost,stats,id,language,flavour,set,finish,edition,sku,number,set.id,set.name,set.released,set.browseable,set.draftable,set.0
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,Fabled,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,863,en,"Beyond the turbulent waters of Death’s Knell, ...",Arcane Rising,Cold foil,{},ARC000-CF,000,arc,Arcane Rising,2020-03-27,True,True,NaN
1,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,Fabled,https://fabdb2.imgix.net/cards/printings/ARC00...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1281,en,"Beyond the turbulent waters of Death’s Knell, ...",Arcane Rising Unlimited,Rainbow foil,{},U-ARC000-RF,000,arc,Arcane Rising,2020-03-27,True,True,NaN


In [37]:
df_unique.to_csv('API_data.csv', index=False)